In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq 

In [ ]:
!sudo add-apt-repository ppa:webupd8team/java
!sudo apt-get update
!sudo apt-get install oracle-java8-installer

 The Oracle JDK License has changed for releases starting April 16, 2019.

The new Oracle Technology Network License Agreement for Oracle Java SE is substantially different from prior Oracle JDK licenses. The new license permits certain uses, such as personal use and development use, at no cost -- but other uses authorized under prior Oracle JDK licenses may no longer be available. Please review the terms carefully before downloading and using this product. An FAQ is available here: https://www.oracle.com/technetwork/java/javase/overview/oracle-jdk-faqs.html

Oracle Java downloads now require logging in to an Oracle account to download Java updates, like the latest Oracle Java 8u211 / Java SE 8u212. Because of this I cannot update the PPA with the latest Java (and the old links were broken by Oracle).

For this reason, THIS PPA IS DISCONTINUED.

UPDATE:

For Oracle Java 17, see a different PPA -> https://www.linuxuprising.com/2021/09/how-to-install-oracle-java-17-lts-on.html

Old descr

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving WSN-DS.csv to WSN-DS.csv


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
# Build the SparkSession
spark = SparkSession.builder \
      .master("local") \
      .appName("MyFirst ML Project") \
      .config("spark.executor.memory", "16gb") \
      .getOrCreate()
    
sc = spark.sparkContext

In [ ]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import warnings
from numpy import concatenate
import numpy as np

In [ ]:
dataset = pd.read_csv("WSN-DS.csv")

In [ ]:
X = dataset.iloc[:, 0 :-1].values # bağımsız değişkenler
y = dataset.iloc[:, -1].values # bağımlı değişken

In [ ]:
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

In [ ]:
dataset_features=pd.DataFrame(X, columns= ['id','Time','Is_CH','whoCH','Dist_To_CH','ADV_S',
                                           'ADV_R','JOIN_S','JOIN_R','SCH_S','SCH_R','Rank',
                                           'DATA_S','DATA_R','Data_Sent_To_BS','dist_CH_To_BS',
                                           'send_code','Expaned Energy'])
dataset_class=pd.DataFrame(y, columns= ['Attack type'])

In [ ]:
dataset_combined_and_labelized =  pd.concat([dataset_features, dataset_class], axis=1)

In [ ]:
dataset_combined_and_labelized.to_csv("dataset_son.csv")

In [ ]:

dataset = spark.read.csv('dataset_son.csv',inferSchema=True, header =True)
dataset = dataset.drop('_c0')

In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Is_CH',
                                       'Dist_To_CH',
                                       'ADV_S',
                                       'ADV_R',
                                       'JOIN_S',
                                       'JOIN_R',
                                       'SCH_S',
                                       'SCH_R',
                                       'Rank',
                                       'DATA_S',
                                       'Data_Sent_To_BS',
                                       'dist_CH_To_BS',
                                       'send_code',
                                       'Expaned Energy'],
                           outputCol='vectorized_features')

final_data = assembler.transform(dataset)

In [ ]:
from pyspark.ml.feature import Normalizer
normalized_dataset=Normalizer().setP(1).setInputCol("vectorized_features").setOutputCol("normalized_features")
normalized_dataset=normalized_dataset.transform(final_data)

In [ ]:
dataset_son=normalized_dataset.select('normalized_features', 'Attack type')

In [ ]:
train_data,test_data = dataset_son.randomSplit([0.7,0.3], seed=0)

In [ ]:
#********************** MULTILAYER PERCEPTRON ALGORİTMASI*******************
layers = [14,6,5] 
from pyspark.ml.classification import MultilayerPerceptronClassifier

mpc = MultilayerPerceptronClassifier(labelCol='Attack type',
                        featuresCol="normalized_features", maxIter=100, layers=layers, blockSize=128, seed=0)
model5 = mpc.fit(train_data)
predictions = model5.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="Attack type", predictionCol= "prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

evaluator = MulticlassClassificationEvaluator(
    labelCol='Attack type', predictionCol="prediction", metricName="f1")
F1_Score = evaluator.evaluate(predictions)
print("F1 Score = %g" % (F1_Score))

evaluator = MulticlassClassificationEvaluator(
    labelCol='Attack type', predictionCol="prediction", metricName="weightedPrecision")
Precision = evaluator.evaluate(predictions)
print("Precision = %g" % (Precision))

evaluator = MulticlassClassificationEvaluator(
    labelCol='Attack type', predictionCol="prediction", metricName="weightedRecall")
Recall = evaluator.evaluate(predictions)
print("Recall = %g" % (Recall))

Accuracy = 0.970387
F1 Score = 0.968514
Precision = 0.970815
Recall = 0.970387


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_true = predictions.select("Attack type")
y_true = y_true.toPandas()

y_pred = predictions.select("prediction")
y_pred = y_pred.toPandas()

target_names = ["Class {}".format(i) for i in range(5)]
print(classification_report(y_true, y_pred, target_names = target_names))

              precision    recall  f1-score   support

     Class 0       0.66      0.91      0.76      2994
     Class 1       0.88      0.91      0.89       996
     Class 2       0.79      0.48      0.60      4358
     Class 3       0.99      1.00      0.99    102471
     Class 4       0.97      0.86      0.91      2036

    accuracy                           0.97    112855
   macro avg       0.86      0.83      0.83    112855
weighted avg       0.97      0.97      0.97    112855



In [ ]:
print(confusion_matrix(y_true, y_pred))

[[  2728     23    243      0      0]
 [    10    907     31     46      2]
 [  1388     13   2090    867      0]
 [    21     89    283 102030     48]
 [    14      0      0    264   1758]]
